В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data = pd.read_csv('orange_small_churn_data.train')
labels = pd.read_csv('orange_small_churn_labels.train', header=None)
#считываем данные

In [3]:
feature = data.isna().sum(axis = 0) == len(data)
zero_features = feature[feature == True].index.tolist()
len(zero_features)
#определим количество полностью пустых столбцов

18

In [4]:
num_features_quan = 0
for feature in zero_features:
    if feature in data.iloc[:, :190].columns:
        num_features_quan += 1
num_features_quan
#определим количество пустых столбцов среди числовых признаков

16

Следовательно после удаления пустых столбцов у нас останется 174 числовых признака и 38 категориальных

In [5]:
data.dropna(how='all', axis=1, inplace=True)
#удалим полностью пустые столбцы

In [6]:
data.shape

(40000, 212)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
#разделим нашу выборку на train и test

In [8]:
X_train.shape

(28000, 212)

In [9]:
X_train.fillna(0, inplace=True)
#заменим все пустые значения на 0

/home/double_dee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [10]:
X_train.iloc[:, 174:] = X_train.iloc[:, 174:].astype('str')
#приведем все значения категориальных признаков к одному типу

/home/double_dee/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
# для преобразования категориальных фич используем LabelEncoder
le = LabelEncoder()
for i in range(174, 212):
    X_train.iloc[:, i] = le.fit_transform(X_train.iloc[:, i])

Для подсчета метрик будем использовать три классификатора с дефолтными настройками:
RidgeClassifier, RandomForestClassifier, GradientBoostingClassifier

## RidgeClassifier

In [13]:
#создание объекта - классификатора
ridge_clf = RidgeClassifier(random_state = 42)

In [14]:
skf = StratifiedKFold()
#т.к анализ показал что целевая переменная не сбалансирована 
#используем стратифицированную кросс-валидацию на 5 фолдов 

/home/double_dee/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:624: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


In [15]:
%time
roc_auc = cross_val_score(ridge_clf, X_train, y_train, cv=skf, scoring='roc_auc', n_jobs=-1).mean()
f1 = cross_val_score(ridge_clf, X_train, y_train, cv=skf, scoring='f1_micro', n_jobs=-1).mean()
accuracy = cross_val_score(ridge_clf, X_train, y_train, cv=skf, scoring='accuracy', n_jobs=-1).mean()
#посмотрим на наши метрики используя кросс-валидацию

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 20.5 µs


In [16]:
print('RidgeClassifier:')
print()
print('ROC_AUC : ', roc_auc)
print('f1 : ', f1)
print('accuracy : ', accuracy)

RidgeClassifier:

ROC_AUC :  0.6557727781724298
f1 :  0.9264642875150703
accuracy :  0.9264642875150703


## RandomForestClassifier

In [17]:
rfc = RandomForestClassifier(random_state=42)

In [18]:
%time
roc_auc = cross_val_score(rfc, X_train, y_train, cv=skf, scoring='roc_auc', n_jobs=-1).mean()
f1 = cross_val_score(rfc, X_train, y_train, cv=skf, scoring='f1_micro', n_jobs=-1).mean()
accuracy = cross_val_score(rfc, X_train, y_train, cv=skf, scoring='accuracy', n_jobs=-1).mean()

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 21.2 µs


In [19]:
print('RandomForestClassifier:')
print()
print('ROC_AUC : ', roc_auc)
print('f1 : ', f1)
print('accuracy : ', accuracy)

RandomForestClassifier:

ROC_AUC :  0.5824963016307755
f1 :  0.9262142824101138
accuracy :  0.9262142824101138


## GradientBoostingClassifier

In [20]:
gbc = GradientBoostingClassifier(random_state=42)

In [21]:
%time
roc_auc = cross_val_score(gbc, X_train, y_train, cv=skf, scoring='roc_auc', n_jobs=-1).mean()
f1 = cross_val_score(gbc, X_train, y_train, cv=skf, scoring='f1_micro', n_jobs=-1).mean()
accuracy = cross_val_score(gbc, X_train, y_train, cv=skf, scoring='accuracy', n_jobs=-1).mean()

CPU times: user 11 µs, sys: 2 µs, total: 13 µs
Wall time: 24.8 µs


In [22]:
print('RandomForestClassifier:')
print()
print('ROC_AUC : ', roc_auc)
print('f1 : ', f1)
print('accuracy : ', accuracy)

RandomForestClassifier:

ROC_AUC :  0.7307263567546715
f1 :  0.9267143079272429
accuracy :  0.9267143079272429


## Вывод:

По основной метрике ROC-AUC лучше всех показал себя градиентный бустинг, по остальным метрикам схожие результаты по всем трем классификаторам.